# Covid and Influenza - sick and death cases comparison

**Part 2 - Analysis**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
url = "https://raw.githubusercontent.com/mborycki/Covid_Influenza_Comparison/main/Covid_and_Influenza.csv"

In [ ]:
df = pd.read_csv(url)

In [ ]:
df.head()

In [ ]:
df.info()

# File preparation / grouping

In [ ]:
# I made a function to have bigger visibility on chart
def CovidCasesDevider(table_name):
    covid_cases_list_mln = ['Confirmed','Deaths','Recovered','Detected_FluCases']

# number of cases decreased by 1000
    for col in covid_cases_list_mln: 
        for value in range(len(table_name)):
            table_name.loc[value,(col)] = table_name.loc[value,(col)] / 1000

# rounded to 2 decimal values            
    for col in covid_cases_list_mln: 
        table_name[col] = table_name[col].apply(lambda x: round(x,2))

# Do not need weeks/year in chart - quarters is enough      
    table_name['YearQuater'] = table_name.Year.astype(str)+'-Q'+table_name.Quarter.astype(str)
    table_name = table_name.drop(["Year","Quarter"],axis=1)
    
    return table_name

In [ ]:
# I made a function to have bigger visibility on chart
def CovidCasesDevider_Country(table_name):
    covid_cases_list_mln = ['Confirmed','Deaths','Recovered','Detected_FluCases']

# number of cases decreased by 1000
    for col in covid_cases_list_mln: 
        for value in range(len(table_name)):
            table_name.loc[value,(col)] = table_name.loc[value,(col)] / 1000

# rounded to 2 decimal values            
    for col in covid_cases_list_mln: 
        table_name[col] = table_name[col].apply(lambda x: round(x,2))

# Do not need weeks/year in chart - quarters is enough      
    table_name['YearQuater'] = table_name.Year.astype(str)+'-Q'+table_name.Quarter.astype(str)
    table_name = table_name.drop(["Year","Quarter"],axis=1)
    
    return table_name

In [ ]:
final_df_quarter = df[['Country', 'Year', 'Confirmed', 'Deaths', 'Recovered',
       'Detected_FluCases', 'Quarter']].groupby(['Country','Year','Quarter']).agg({'Confirmed':'max', 'Deaths':'max', 'Recovered':'max',
       'Detected_FluCases':'sum'}).sort_values(['Country','Year','Quarter']).reset_index()
final_df_quarter.head()

# Analysis

In [ ]:
# for all countries - TOTAL
chart_1 = CovidCasesDevider(final_df_quarter.groupby(['Year','Quarter'])\
.agg({'Confirmed':'sum', 'Deaths':'sum', 'Recovered':'sum','Detected_FluCases':'sum'})\
.sort_values(['Year','Quarter']).reset_index())

In [ ]:
chart_1.head(2)

In [ ]:
# for chosen country
chosen_country = 'Poland'

chart_2 = CovidCasesDevider(final_df_quarter[final_df_quarter.Country==chosen_country].groupby(['Year','Quarter']).\
agg({'Confirmed':'sum', 'Deaths':'sum', 'Recovered':'sum','Detected_FluCases':'sum'}).\
sort_values(['Year','Quarter']).reset_index())

In [ ]:
chart_2.head(2)

In [ ]:
fig, ax1 = plt.subplots()
plt.rcParams["figure.figsize"] = (25,15)
plt.xticks(fontsize=16, rotation=45)
plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)
# plt.ylim([-3, 3])

y = chart_1['Confirmed']
x = chart_1.YearQuater.unique()
z = chart_1.Detected_FluCases

ax1.set_title(f"Confirmed Covid Cases per week ('000)", fontsize=28)

color = 'tab:red'
ax1.set_xlabel('Periods', fontsize=24)
ax1.set_ylabel("Confirmed Covid19 Cases ('000)", color=color, fontsize=24)
ax1.plot(x, y, color=color)
ax1.tick_params(axis='y', labelcolor=color, labelsize=16) 
# ax1.set_ylim(0,60000)

for xx,yy in zip(x,y):
    label = "{:.0f}".format(yy)
    plt.annotate(label, (xx,yy), textcoords="offset points", xytext=(0,10), ha='right', fontsize=20) 

ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel("Detected Inluenza Cases ('000)", color=color, fontsize=24)  # we already handled the x-label with ax1
ax2.plot(x, z, color=color)
ax2.tick_params(axis='y', labelcolor=color, labelsize=16)
# ax2.set_ylim(0,2000)
# top_lim = 2000
# ax2.set_ylim(top=top_lim)

for xx,zz in zip(x,z):
    # label = f'{round(zz, 0):,}'
    label = "{:.0f}".format(zz)
    plt.annotate(label,(xx,zz),textcoords="offset points",xytext=(0,10),ha='right', fontsize=20)


fig.tight_layout()  
plt.show()